### notes for code

In [55]:
import random
import math as m
import numpy as np
from numpy.linalg import norm
import scipy
import sympy
import scipy.integrate as spint
from sympy import diff, symbols
import matplotlib.pyplot as plt

class MFA:
    def __init__(self):
        #expectation values; choosing randomly starting value
#         self.l = random.uniform(-1,1)  #lambda
#         self.t = random.uniform(-1,1)  #t
        #given parameters
        self.a = -0.5  #alpha
        self.b = 0.5 #beta
        
        
    def E(self, l , t , a ,b , low = 0, high = 2*np.pi, skips=100): #works
        k = np.linspace(low, high, skips)
        integral = -1/(4*np.pi) * (b**2*(1+2*l*(1+a))**2 + (1-2*t*(1+a))**2 + 2*b*(1+2*l*(1+a)) * (1-2*t*(1+a)) * np.cos(k))**(1/2) 
#         print(integral)
        res = spint.simps(integral, k)
        return res + (1+a)*(b*l**2 - t**2)     
    
    # from mathematica
    
    def delE_l(self, l, t, a, b, low = 0, high = 2*np.pi, skips =100):
        k = np.linspace(low, high, skips)
        term1 = 2*(1 + a) *b*l 
        integral = (2*(1+a)*b*l*(1 +(1 + a)*l**2))/(8*np.pi*np.sqrt((1 - 2*t*(1 + a))**2 + 2*np.cos(k)*(1 - 2*t*(1 + a)) * b* (1 + 2*(1 + a) *l) + b**2 *(1 + (1 + a)*l**2)**2))
#         print(integral)
        res = spint.simps(integral, k)
#         print(res)
        return term1 - res     
# 
    def delE_t(self, l, t, a, b, low = 0, high = 2*np.pi, skips =100):
        k = np.linspace(low, high, skips)
        term1 = -2*t*(1+a) 
        integral = (-4*(1 + a)*(1 - 2*t * (1 + a)) - 4*np.cos(k)*(1+a)*b*(1 + 2 * (1 + a)*l)) / (8*np.pi*np.sqrt((1 - 2*t*(1 + a))**2 + 2*np.cos(k)*(1 - 2*t*(1 + a)) * b* (1 + 2*(1 + a) *l) + b**2* (1 + (1 + a)*l**2)**2))
        res = spint.simps(integral, k)
#         print(res)
        return term1 - res    

    def gradE(self, l, t, a, b):
        return np.array([self.delE_l(l, t, a, b), self.delE_t(l, t, a, b)])
    
#     def SteepDescent(self, gradients, learning_rate = 0.001):
#         self.l -= learning_rate*gradients[0]
#         self.t -= learning_rate*gradients[1]

def solve_sys1(l_start, t_start, a, b,steps = 0.01, tolerance = 3):
    mfa = MFA()
    # initial system
    l0 = l_start
    t0 = t_start
#     print(l0, t0)
    # loop / recurse till local/global minimum
    return solving_loop(l0, t0, a, b, steps, tolerance)

#with reference to: https://www.cs.toronto.edu/~guerzhoy/411/lec/W02/python/graddescent2d.html

def descent(l,t,a,b, grads, learn_rate, maxloop, tolerance):
    mfa = MFA()
#     E = mfa.E(l,t,a,b)
#     epochs = 10**(-5)
    res = np.array([l,t])
#     t_initial = init_sys - 10*epochs
    
#     res = init_sys.copy()

    for i in range(maxloop):
#         init_sys = res.copy()
#         print(type(learn_rate),type(grads(res[0], res[1],a,b)))
        
        delE_l = mfa.delE_l(l, t, a, b)
        delE_t = mfa.delE_t(l, t, a, b)
#         if round(delE_l,tolerance) != 0 and round(delE_t,tolerance) != 0:
        res -= learn_rate*grads(res[0], res[1],a,b)
#         if grads(l, t,a,b)[0] > 0:
#             res[0] -= learn_rate * grads(l, t,a,b)[0]
#         elif grads(l, t,a,b)[0] < 0:
#             res[0] += learn_rate * grads(l, t,a,b)[0]
#         if grads(l, t,a,b)[1] > 0:
#             res[1] -= learn_rate * grads(l, t,a,b)[1]
#         elif grads(l, t,a,b)[1] < 0:
#             res[1] += learn_rate * grads(l, t,a,b)[1]
#         print([res[0],res[1]], mfa.E(res[0],res[1], a,b), grads(res[0],res[1], a, b))
#         else:
#             break
    return [res[0],res[1]]
#         return None
#        
        
#         if round(grads(res[0], res[1],a,b)[0],tolerance) == 0 and round(grads(res[0], res[1],a,b)[1],tolerance)==0:
#             break
#             return res
#         else:
    return [l,t]
# ,l_list,t_list,E_list

# def recurse_sol(func, l,t,a,b, grads, learn_rate, maxloop, tolerance):
#     mfa = MFA()
#     init_sys = np.array([l,t])
#     E = func(l,t,a,b) - learn_rate * grads(l,t,a,b)
#     i


def solve_sys2(l, t, a, b, learn_rate, maxloop, tolerance):
    mfa = MFA()
    print(l, t)
#     E = mfa.E(l,t,a,b)
#     for i in range(maxloop):
#         mfa.SteepDescent(mfa.gradE(l, t, a, b), learning_rate = learn_rate) 
#     return [l,t]
    return descent(l, t, a, b, mfa.gradE, learn_rate, maxloop, tolerance)

def plot_sys2(l_list, t_list, E_list):
    mfa = MFA()
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.contour3D(t_list, l_list, E_list, cmap='binary')
    ax.set_xlabel('t')
    ax.set_ylabel('lambda')
    ax.set_zlabel('epsilon')

def main():
    mfa = MFA()
    a = mfa.a
    b = mfa.b
    l = random.gauss(0,3)
    t = random.gauss(0,3)
    result = solve_sys2(l, t, a, b, learn_rate = 0.01, maxloop = 1000, tolerance = 5)
    print(result)
#     plotL = np.linspace(-1,1,100)
#     plott = np.linspace(-1,1,100)
#     L, T = np.meshgrid(plotL, plott)
#     E = mfa.E(L, T, a, b)
#     plot_sys2(L, T, E)
    

def test():
    mfa = MFA()
    tlist = np.linspace(0.1,1.0,0.001)
    print(tlist)
#     for t in tlist:
# #     print(mfa.E(1.0, 0.5, -0.2, 0.5))
# #     print(mfa.delE_l(1.0, 0.5, -0.3, 0.5))
#         print(mfa.delE_t(1.0, t, -0.3, 0.5))
#         print(mfa.gradE(1.0, t, -0.3, 0.5))

In [56]:
for i in range(20):
    main()

-3.7030944609388534 -1.1164171329154398
[-0.03199862980530273, -31462.52313307257]
-5.356354076560179 0.4618116626433543
[-0.06705297100727703, 11253.31334766033]
4.397531756136818 -1.2478858766193668
[0.040021206869376866, -33447.18109637347]
1.1188279807279942 2.949766454141569
[0.007983429208202135, 72320.12649327295]
3.9718599185086956 -1.7534019763661925
[0.033524522178522254, -45196.63468570582]
-0.5670899050790266 -2.3529827382772037
[-0.003943668242755279, -59748.302573827175]
-5.986832073515087 2.41086755395576
[-0.056501313930019864, 56357.16647696267]
-2.11671987721974 4.829487906116369
[-0.014978418660475152, 111518.8891063393]
4.724777211791021 -1.7247725910460217
[0.041888180290334164, -43741.05613832845]
-2.553132955077714 -2.656188110890476
[-0.018733024866241493, -65760.15731056282]
0.7310939890347087 2.214092043817383
[nan, nan]

C:\Users\ryanl\AppData\Local\Temp/ipykernel_37204/1924971915.py:33: RuntimeWarning: invalid value encountered in sqrt
  integral = (2*(1+a)*b*l*(1 +(1 + a)*l**2))/(8*np.pi*np.sqrt((1 - 2*t*(1 + a))**2 + 2*np.cos(k)*(1 - 2*t*(1 + a)) * b* (1 + 2*(1 + a) *l) + b**2 *(1 + (1 + a)*l**2)**2))
C:\Users\ryanl\AppData\Local\Temp/ipykernel_37204/1924971915.py:42: RuntimeWarning: invalid value encountered in sqrt
  integral = (-4*(1 + a)*(1 - 2*t * (1 + a)) - 4*np.cos(k)*(1+a)*b*(1 + 2 * (1 + a)*l)) / (8*np.pi*np.sqrt((1 - 2*t*(1 + a))**2 + 2*np.cos(k)*(1 - 2*t*(1 + a)) * b* (1 + 2*(1 + a) *l) + b**2* (1 + (1 + a)*l**2)**2))



1.8330206250774252 -0.24007093665973422
[nan, nan]
-1.8962832077444447 -3.272833071386171
[-0.013397154647419765, -78937.70481111035]
1.5670932746397046 -1.3777568527148738
[0.011572095436489155, -39294.490534711134]
-0.23212303524666722 3.7697313141074043
[-0.001603058022833858, 89456.02187923303]
4.754968123477071 3.9294671377925683
[0.03918451612890141, 90679.95401329007]
-3.047124484758686 -2.2833356011552155
[-0.023239567850827675, -57564.390456189714]
-2.5178802071256308 -0.3433201353718285
[-0.022061664692726395, -15451.705767149027]
3.0446350416959427 0.8272747050995382
[0.031262904384805, 21386.240444698753]
2.250134200899632 0.7311503144469054
[nan, nan]



Bin area

In [3]:
#     def nume_integrate(self, func, low, high, skips = 1000):
#         h = 1/float(skips) 
#         k_list = list(np.linspace(low,high,1000))
        
#         for k in k_list:
#             func += (func(k)+func(k+h))*h/2
#         return func
    
#     def E(self, l ,t, a, b, k, skips = 1000, low = 0, high = 2): # energy density
#         e(k) = self.integrand(l , t, a ,b , k) + (1+a)*(b*l**2 - t**2)
#         h = 1/float(skips) 
#         k_list = list(np.linspace(low,high,skips))
#         for kval in k_list:
#             e += (e(kval) + e(kval + h))*h/2
#         return self.nume_integrate(e, 0,2*m.pi)

#     def check_consistency(func, tolerance = 3): ## function, decimal points rounded
#         if round(func, 3) == 0:
#             return True
#         return False
#     def E(self, l , t , a ,b , low = 0, high = 2*m.pi, skips=100):
#         k = np.linspace(low, high, skips+1)
#         integral = (b**2*(1+2*l*(1+a))**2 + (1-2*t*(1+a))**2 + 2*b*(1+2*l*(1+a)) * (1-2*t*(1+a)) * np.cos(k))**(1/2) + (1+a)*(b*l**2 - t**2)
#         res = spint.simps(integral, k)
#         return -1/(4*np.pi) * res 

# def solving_loop(l,t,a,b,steps = 0.01, tolerance = 4):
#     mfa = MFA()
#     delE_l = mfa.delE_l(l, t, a, b)
#     delE_t = mfa.delE_t(l, t, a,b)
 
#     if round(delE_l,tolerance) != 0 and round(delE_t,tolerance) != 0:
#         if delE_l > 0:
#             l -= steps
#             if delE_t > 0:
#                 t -= steps
#             elif delE_t < 0:
#                 t += steps
                
#         elif delE_l < 0:
#             l += steps
#             if delE_t > 0:
#                 t -= steps
#             elif delE_t < 0:
#                 t += steps
#         return solving_loop(l ,t, a,b,steps = 0.01, tolerance = 4)
    
#     else:
#         return [round(l,tolerance) , round(t,tolerance)]
    